In [166]:
import numpy as np
from mindquantum.core.gates import RX, RY, RZ, H, X, Y, Z, CNOT
from mindquantum.core.circuit import Circuit
import mindspore as ms
from mindquantum.simulator import  Simulator
from mindquantum.core.gates import GroupedPauli
from mindquantum.core.operators import TimeEvolution,QubitOperator
from mindquantum.core.parameterresolver import PRConvertible,PRGenerator,ParameterResolver
from DQAS_tool import generate_pauli_string,one_hot
from mindquantum.core.gates import RotPauliString
from mindquantum.core.gates import UnivMathGate
from mindspore import Tensor, ops
from mindquantum.core.circuit import UN
from mindquantum.core.operators import Hamiltonian             # 导入Hamiltonian模块，用于构建哈密顿量
from mindquantum.framework import MQLayer
from mindspore.nn import  TrainOneStepCell
from mindspore.nn import SoftmaxCrossEntropyWithLogits                         # 导入SoftmaxCrossEntropyWithLogits模块，用于定义损失函数
from mindspore.nn import Adam                                                  # 导入Adam模块用于定义优化参数
from mindspore.train import Accuracy, Model, LossMonitor                       # 导入Accuracy模块，用于评估预测准确率
import mindspore as ms
from mindspore import Parameter, Tensor
from mindspore.dataset import NumpySlicesDataset
from torch.utils.data import DataLoader# 导入NumpySlicesDataset模块，用于创建模型可以识别的数据集
import sys
sys.path.append('..')
from data_processing import X_train,X_test,y_train,y_test
num_layer = 3
# 定义标准差和形状
stddev = 0.02
shape_nnp = (num_layer, 8)
shape_stp = (num_layer, 12)

rtype = np.float64
ctype = np.complex128
# 使用 numpy 生成随机数矩阵
nnp = np.random.normal(loc=0.0, scale=stddev, size=shape_nnp).astype(rtype)
stp = np.random.normal(loc=0.0, scale=stddev, size=shape_stp).astype(rtype)
#Operator Pool
unbound_opeartor_pool = [generate_pauli_string(n=8,seed=i)[0] for i in range(8)]
bound_opeartor_pool = [generate_pauli_string(n=8,seed=i)[1] for i in range(8,12)]

In [193]:
def DQAS_ansatz(Structure_p:np.array,Ansatz_p:np.array,n_layer:int,n_qbits:int=8):
    """
    和 DQAS 文章描述的一致，生成权重线路
    """
    
    Structure_p = Parameter(Tensor(Structure_p, ms.float32), requires_grad=True)
    Ansatz_p = Parameter(Tensor(Ansatz_p, ms.float32), requires_grad=True)
    softmax = ops.Softmax()
    my_stp = softmax(Tensor(Structure_p, ms.float32))
    ansatz = Circuit()
    
    pr_gen = PRGenerator(name='stp*nnp')
    pr_gen2 = PRGenerator(name='stp')

    for i in range(n_layer):
        
        paramertized_part_count=0
        for index_op,each_op in enumerate(unbound_opeartor_pool):
            ansatz += TimeEvolution(QubitOperator(terms=each_op,coefficient=pr_gen.new()),time=1.0).circuit
            paramertized_part_count+=1
            
        for index_op,each_op in enumerate(bound_opeartor_pool):
            #print(index_op,each_op,paramertized_part_count)
            op = GroupedPauli(each_op)
            tmp_cir = Circuit([GroupedPauli(each_op).on(range(n_qbits))])
            matrix = tmp_cir.matrix()
            #print(matrix.shape,stp[i,index_op+paramertized_part_count])
            ansatz += UnivMathGate(matrix_value=matrix*pr_gen2.new(),name=op.pauli_string).on(range(n_qbits))  
    return ansatz
    

In [195]:
def DQAS_ansatz2(Structure_p:np.array,Ansatz_p:np.array,n_layer:int,n_qbits:int=8):
    """
    和 DQAS 文章描述的一致，生成权重线路
    """
    
    Structure_p = Parameter(Tensor(Structure_p, ms.float32), requires_grad=True)
    Ansatz_p = Parameter(Tensor(Ansatz_p, ms.float32), requires_grad=True)
    softmax = ops.Softmax()
    my_stp = softmax(Tensor(Structure_p, ms.float32))
    ansatz = Circuit()
    
    pr_gen = PRGenerator(name='stp*nnp')
    pr_gen2 = PRGenerator(name='stp')

    for i in range(n_layer):
        
        paramertized_part_count=0
        for index_op,each_op in enumerate(unbound_opeartor_pool):
            ansatz += TimeEvolution(QubitOperator(terms=each_op,coefficient=pr_gen.new()),time=1.0).circuit
            paramertized_part_count+=1
            
        for index_op,each_op in enumerate(bound_opeartor_pool):
            #print(index_op,each_op,paramertized_part_count)
            op = GroupedPauli(each_op)
            tmp_cir = Circuit([GroupedPauli(each_op).on(range(n_qbits))])
            matrix = tmp_cir.matrix()
            #print(matrix.shape,stp[i,index_op+paramertized_part_count])
            ansatz += UnivMathGate(matrix_value=matrix*pr_gen2.new().values,name=op.pauli_string).on(range(n_qbits))  
    return ansatz
    

In [ ]:
def Ansatz_loss(Ansatz:Circuit):
    '''本函数是 测定给定的Ansatz的 QNN 二分类损失函数值 并给出关于Ansatz的梯度
    
    '''
    prg = PRGenerator('alpha')
    nqbits = 8
    encoder = Circuit()
    encoder += UN(H, nqbits)                                  
    for i in range(nqbits):                                   
        encoder += RY(prg.new()).on(i)                 
    encoder = encoder.no_grad()
    encoder = encoder.as_encoder()
    hams = [Hamiltonian(QubitOperator(f'Z{i}')) for i in [0,1]]
    #ansatz = ansatz.as_ansatz()
    ms.set_context(mode=ms.PYNATIVE_MODE, device_target="CPU")
    circuit = encoder+ Ansatz.as_ansatz()    
    sim = Simulator('mqvector', n_qubits=nqbits)
    sim.get_expectation(hamiltonian=hams[0],circ_left=circuit)
    
    

In [196]:
DQAS_ansatz2(stp,nnp,3,8)

TypeError: unsupported operand type(s) for *: 'complex' and 'method'

In [198]:
class ForwardAndLoss(ms.nn.Cell):
    def __init__(self, backbone, loss_fn):
        super(ForwardAndLoss, self).__init__(auto_prefix=False)
        self.backbone = backbone
        self.loss_fn = loss_fn

    def construct(self, data, label):
        output = self.backbone(data)
        return self.loss_fn(output, label)

    def backbone_network(self):
        return self.backbone
    
    
class TrainOneStep(ms.nn.TrainOneStepCell):

    def __init__(self, network, optimizer):
        super(TrainOneStep, self).__init__(network, optimizer)
        self.grad = ms.ops.GradOperation(get_by_list=True)

    def construct(self, data, label):
        weights = self.weights
        loss = self.network(data, label)
        grads = self.grad(self.network, weights)(data, label)
        return loss, self.optimizer(grads)


In [199]:
def Mindspore_ansatz(Structure_p:Parameter,Ansatz_p:Parameter,n_layer:int,n_qbits:int=8):
    """
    和 DQAS 文章描述的一致，生成权重线路
    Structure_p:np.array DQAS中的权重参数,
    Ansatz_p:np.array  DQAS中的Ansatz参数,
    
    """
    # Structure_p = Parameter(Tensor(Structure_p, ms.float32), requires_grad=True)
    # Ansatz_p = Parameter(Tensor(Ansatz_p, ms.float32), requires_grad=True)
    softmax = ops.Softmax()
    my_stp = softmax(Tensor(Structure_p, ms.float32))
    ansatz = Circuit()

    for i in range(n_layer):
        
        paramertized_part_count=0
        for index_op,each_op in enumerate(unbound_opeartor_pool):

            # ansatz_param = Ansatz_p[i,index_op]
            #Structure_param =float(stp[i,index_op])
            ansatz += TimeEvolution(QubitOperator(terms=each_op,coefficient=float(Ansatz_p[i,index_op])),time=float(my_stp[i,index_op])).circuit
            paramertized_part_count+=1
            
        for index_op,each_op in enumerate(bound_opeartor_pool):
            #print(index_op,each_op,paramertized_part_count)
            op = GroupedPauli(each_op)
            tmp_cir = Circuit([GroupedPauli(each_op).on(range(n_qbits))])
            matrix = tmp_cir.matrix()
            #print(matrix.shape,stp[i,index_op+paramertized_part_count])
            ansatz += UnivMathGate(matrix_value=matrix*float(my_stp[i,index_op+paramertized_part_count]),name=op.pauli_string).on(range(n_qbits))  
            
    # Encoder
    prg = PRGenerator('alpha')
    nqbits = n_qbits
    encoder = Circuit()
    encoder += UN(H, nqbits)                                  
    for i in range(nqbits):                                   
        encoder += RY(prg.new()).on(i)                 
    encoder = encoder.no_grad()
    encoder = encoder.as_encoder()
    hams = [Hamiltonian(QubitOperator(f'Z{i}')) for i in [0,1]]
    #ansatz = ansatz.as_ansatz()
    ms.set_context(mode=ms.PYNATIVE_MODE, device_target="CPU")
    #ansatz = DQAS_ansatz(Structure_p=stp,Ansatz_p=nnp,n_layer=3,n_qbits=8)
    circuit = encoder+ ansatz.as_ansatz()    
    sim = Simulator('mqvector', n_qubits=nqbits)
    f = sim.get_expectation(hams[0],circuit,pr=X_train)
    
    grad_ops= sim.get_expectation_with_grad(hams,circuit)
    QuantumNet = MQLayer(grad_ops)          # 搭建量子神经网络
    
    



In [190]:
cir = Mindspore_ansatz(Structure_p=stp,Ansatz_p=nnp,n_layer=3,n_qbits=8)

In [177]:
from mindspore import grad




Stp = Parameter(Tensor(stp, ms.float32), requires_grad=True)
Nnp = Parameter(Tensor(nnp, ms.float32), requires_grad=True)


# 确保 wrapped_mindspore_ansatz 返回的是标量
def wrapped_mindspore_ansatz(Structure_p, Ansatz_p, n_layer=3, n_qbits=8):
    loss_value = Mindspore_ansatz(Structure_p, Ansatz_p, n_layer=n_layer, n_qbits=n_qbits)
    print("Inside Mindspore_ansatz - Loss Value Shape:", loss_value.shape)
    return loss_value # 确保返回标量

# 计算损失
loss_value = Mindspore_ansatz(Stp, Nnp, n_layer=3, n_qbits=8)

# 计算梯度
grad_structure = grad(wrapped_mindspore_ansatz, grad_position=0)(stp, nnp)
grad_ansatz = grad(wrapped_mindspore_ansatz, grad_position=1)(stp, nnp)

print("Loss Value:", loss_value)
print("Loss Value Shape:", loss_value.shape)  # 确保这是标量
print("Stp Shape:", Stp.shape)
print("Nnp Shape:", Nnp.shape)

print("Gradient for Structure_p:", grad_structure)
print("Gradient for Ansatz_p:", grad_ansatz)


Loss Value: 0.69315296
Gradient for Structure_p: ()
Gradient for Ansatz_p: ()


In [179]:
from mindspore import grad, Tensor, Parameter, ops

# 创建参数
Stp = Parameter(Tensor(stp, ms.float32), requires_grad=True)
Nnp = Parameter(Tensor(nnp, ms.float32), requires_grad=True)

# 确保 wrapped_mindspore_ansatz 返回的是标量
def wrapped_mindspore_ansatz(Structure_p, Ansatz_p, n_layer=3, n_qbits=8):
    loss_value = Mindspore_ansatz(Structure_p, Ansatz_p, n_layer=n_layer, n_qbits=n_qbits)
    return loss_value  # 确保返回标量

# 计算损失
loss_value = wrapped_mindspore_ansatz(Stp, Nnp, n_layer=3, n_qbits=8)

# 计算梯度
grad_structure = grad(wrapped_mindspore_ansatz, grad_position=0)(Stp, Nnp)
grad_ansatz = grad(wrapped_mindspore_ansatz, grad_position=1)(Stp, Nnp)

print("Loss Value:", loss_value)
print("Gradient for Structure_p:", grad_structure)
print("Gradient for Ansatz_p:", grad_ansatz)


Loss Value: 0.69315296
Gradient for Structure_p: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Gradient for Ansatz_p: [[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]


In [192]:
from mindspore import value_and_grad

# 定义计算损失和梯度的函数
def loss_and_grad(Structure_p, Ansatz_p, n_layer=3, n_qbits=8):
    loss_value = Mindspore_ansatz(Structure_p, Ansatz_p, n_layer=n_layer, n_qbits=n_qbits)
    return loss_value

# 计算损失值和梯度
# value_and_grad(loss_and_grad, (0, 1))(stp, nnp)

# print("Loss Value:", loss_value)
# print("Gradient for Structure_p:", grad_structure)
# print("Gradient for Ansatz_p:", grad_ansatz)


(      ┏━━━┓ ┏━━━━━━━━━━━━┓                                              ┏━━━━━━━━━┓    
 q0: ──┨ H ┠─┨ RY(alpha0) ┠──────────────────────────────────────────────┨ RX(π/2) ┠─[red bold]↯[/]─ 
       ┗━━━┛ ┗━━━━━━━━━━━━┛                                              ┗━━━━━━━━━┛    
       ┏━━━┓ ┏━━━━━━━━━━━━┓                                              ┏━━━┓          
 q1: ──┨ H ┠─┨ RY(alpha1) ┠──────────────────────────────────────────────┨ H ┠───────[red bold]↯[/]─ 
       ┗━━━┛ ┗━━━━━━━━━━━━┛                                              ┗━━━┛          
       ┏━━━┓ ┏━━━━━━━━━━━━┓ ┏━━━┓                                        ┏━━━┓          
 q2: ──┨ H ┠─┨ RY(alpha2) ┠─┨ H ┠───■────────────────────────────────■───┨ H ┠───────[red bold]↯[/]─ 
       ┗━━━┛ ┗━━━━━━━━━━━━┛ ┗━━━┛   ┃                                ┃   ┗━━━┛          
       ┏━━━┓ ┏━━━━━━━━━━━━┓       ┏━┻━┓                            ┏━┻━┓ ┏━━━┓          
 q3: ──┨ H ┠─┨ RY(alpha3) ┠───────┨╺╋╸┠───■────────────────────■───┨╺╋╸

In [182]:
stp.shape

(3, 12)

In [120]:
ansatz = DQAS_ansatz(Structure_p=stp,Ansatz_p=nnp,n_layer=3,n_qbits=8)

In [122]:
# Encoder
prg = PRGenerator('alpha')
nqbits = 8
encoder = Circuit()
encoder += UN(H, nqbits)                                  # H门作用在每1位量子比特
for i in range(nqbits):                                   # i = 0, 1, 2, 3
    encoder += RY(prg.new()).on(i)                 # RZ(alpha_i)门作用在第i位量子比特
encoder = encoder.no_grad()
encoder = encoder.as_encoder()# Encoder作为整个量子神经网络的第一层，不用对编码线路中的梯度求导数，因此加入no_grad()

hams = [Hamiltonian(QubitOperator(f'Z{i}')) for i in [0,1]]
#ansatz = ansatz.as_ansatz()
ms.set_context(mode=ms.PYNATIVE_MODE, device_target="CPU")
ansatz = DQAS_ansatz(Structure_p=stp,Ansatz_p=nnp,n_layer=3,n_qbits=8)
circuit = encoder+ ansatz.as_ansatz()    
sim = Simulator('mqvector', n_qubits=nqbits)

obs = sim.get_expectation(hamiltonian=hams[0],circ_left=circuit,pr=nnp.reshape(-1))
# grad_ops = sim.get_expectation_with_grad(hams,
#                                         circuit,
#                                         parallel_worker=5)

# f,g1,g2 = grad_ops(X_train,nnp.reshape(-1))
# tensor_f = Tensor(f,ms.float32)
# loss_fn = SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')  
# # all_data = Tensor(X_train,ms.float32) 
# all_label = Tensor(y_train,ms.int32)
# loss_value = loss_fn(tensor_f,all_label) 




ValueError: data requires a number or a ParameterResolver or a dict or a string, but get <class 'numpy.ndarray'>

In [109]:
loss_value

Tensor(shape=[], dtype=Float32, value= 0.693153)

In [28]:
f,g1,g2 = grad_ops(X_train[0],nnp.reshape(-1))
tensor_f = Tensor(f,ms.float32) #Tensor(shape=[1, 2], dtype=Float32
label = Tensor(y_train[0],ms.int32) #Tensor(shape=[], dtype=Int32, value= 0)
tensor_label = one_hot(label,num_classes=2)
expand_dims_op = ops.ExpandDims()
expanded_label = expand_dims_op(tensor_label, 0)

loss_fn = SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')  
# loss_fn(tensor_f,one_hot(label,num_classes=2))

In [29]:
expanded_label

Tensor(shape=[1, 2], dtype=Int32, value=
[[1, 0]])

In [31]:
loss_fn(tensor_f,tensor_label)

ValueError: Labels shape value must be equal to the Features except the last dimension of Features

----------------------------------------------------
- C++ Call Stack: (For framework developers)
----------------------------------------------------
mindspore/core/ops/sparse_softmax_cross_entropy_with_logits.cc:49 SparseSoftmaxCrossEntropyWithLogitsInferShape


In [14]:
f,g1,g2 = grad_ops(X_train,nnp.reshape(-1)) 
tensor_f = Tensor(f,ms.float32) #Tensor(shape=[963, 2]
loss_fn = SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')  
all_data = Tensor(X_train,ms.float32) 
all_label = one_hot(Tensor(y_train,ms.int32),num_classes=2) #Tensor(shape=[963, 2]
# # one_hot(labels = Tensor([0], ms.int32), num_classes=2

In [15]:
tensor_f

Tensor(shape=[963, 2], dtype=Float32, value=
[[ 6.10233330e-27, -9.42179412e-27],
 [ 1.16670155e-26,  1.54411310e-26],
 [ 5.37693525e-27, -1.05122965e-26],
 ...
 [ 1.11175392e-26, -1.63111136e-26],
 [-1.44161827e-28,  1.09453526e-26],
 [-1.97711461e-27,  9.93502171e-30]])

In [13]:
loss_fn(tensor_f,all_label)

ValueError: Labels shape value must be equal to the Features except the last dimension of Features

----------------------------------------------------
- C++ Call Stack: (For framework developers)
----------------------------------------------------
mindspore/core/ops/sparse_softmax_cross_entropy_with_logits.cc:49 SparseSoftmaxCrossEntropyWithLogitsInferShape


In [ ]:
all_data

In [ ]:
one_hot(labels = Tensor(y_train[0], ms.int32), num_classes=2)

In [ ]:
import mindspore as ms
from mindspore import Tensor, ops, nn
from mindspore.ops import composite as C
import numpy as np

# 定义一个简单的函数
def f(x, y):
    return x**2 + 2*y

# 包装函数以便计算值和梯度
def value_and_grad(f, argnums=(0,)):
    def wrapper(*args):
        # 将输入转换为 MindSpore 的 Tensor
        ms_args = [Tensor(arg, dtype=ms.float32) for arg in args]
        
        # 定义计算梯度的函数
        def grad_fn(*ms_args):
            return f(*ms_args)
        
        # 计算函数值
        value = f(*ms_args)
        
        # 计算梯度
        grads = C.GradOperation(get_all=True, sens_param=False)(grad_fn)(*ms_args)
        
        # 提取指定参数的梯度
        selected_grads = [grads[i] for i in argnums]
        
        return value.asnumpy(), [g.asnumpy() for g in selected_grads]
    
    return wrapper

# 示例使用
g = value_and_grad(f)
value, grad = g(np.array(1.0), np.array(2.0))
print("Value:", value)  # 输出: 5.0
print("Grad:", grad)    # 输出: [2.0]

g = value_and_grad(f, argnums=(0, 1))
value, grads = g(np.array(1.0), np.array(2.0))
print("Value:", value)  # 输出: 5.0
print("Grads:", grads)  # 输出: [2.0, 2.0]

In [150]:
pr_nnp = PRGenerator(name='nnp')
pr_stp = PRGenerator(name='stp')
TimeEvolution(QubitOperator(terms=unbound_opeartor_pool[0], coefficient=pr_nnp.new()), time=1.0).circuit


┏━━━┓                                        ┏━━━┓   
q2: ──┨ H ┠───■────────────────────────────────■───┨ H ┠───
      ┗━━━┛   ┃                                ┃   ┗━━━┛   
            ┏━┻━┓                            ┏━┻━┓         
q3: ────────┨╺╋╸┠───■────────────────────■───┨╺╋╸┠─────────
            ┗━━━┛   ┃                    ┃   ┗━━━┛         
                  ┏━┻━┓ ┏━━━━━━━━━━━━┓ ┏━┻━┓               
q6: ──────────────┨╺╋╸┠─┨ RZ(2*nnp0) ┠─┨╺╋╸┠───────────────
                  ┗━━━┛ ┗━━━━━━━━━━━━┛ ┗━━━┛

In [148]:
unbound_opeartor_pool[0]

'I0 I1 X2 Z3 I4 I5 Z6 I7'